# Parallel teleport

Start by initializing the environment

In [ ]:
from qdk import init, TargetProfile, code
from qdk.qsharp import compile, circuit, eval
from qdk.simulation import AC1000, NoiseConfig
from qdk.widgets import Circuit, Histogram

init(target_profile=TargetProfile.Base)

## Teleportation

Load the teleportation program from the Q\# file

In [ ]:
from pathlib import Path
eval(Path("teleport-layout.qs").read_text(encoding='utf-8'))

## Circuit

Verify the circuit for two rounds of teleportation

In [ ]:
Circuit(circuit(code.Layout, 2))

## Visualization

Now run 120 rounds of teleportation in parallel and visualize

In [ ]:
# Compile the code to the high-level QIR instructions
qir = compile(code.Layout, 120)

# Visualize a trace of running the QIR on the target machine
ac1000 = AC1000()
ac1000.trace(qir)

## Clifford simulation

Run simulations for 1,000 shots. First write a helper so the historgrams are readable

In [ ]:
from histogram_helper import expect_zeros_histogram

results = ac1000.simulate(qir, shots=1000)
Histogram(expect_zeros_histogram(results))

In [ ]:
# Now with noise.
noise = NoiseConfig()
noise.sx.loss = 0.001
results = ac1000.simulate(qir, shots=1000, noise=noise)
Histogram(expect_zeros_histogram(results))

# Submit the job to Azure Quantum

In [ ]:
from qdk.azure import Workspace
from qdk.azure.target import Target

# Access workspace via Microsoft corp account
workspace = Workspace(
    subscription_id = "677fc922-91d0-4bf6-9b06-4274d319a0fa",
    resource_group = "rg-atom-partner-access-wus2",
    name = "qw-atom-devtools-wus",
    location = "westus",
)

target = workspace.get_targets("atom.sim.ac1000.physical")

if not isinstance(target, Target) or target.current_availability != "Available":
    raise RuntimeError("Target not available")

job = target.submit(qir, "demo1", input_params={"shots": 100})
results = job.get_results()
Histogram(results)